# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings

# Import API key
from api_keys import geoapify_key

warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,46.49,55,0,9.22,US,1732486790
1,1,hamilton,39.1834,-84.5333,54.77,64,0,9.22,US,1732486837
2,2,suva,-18.1416,178.4415,81.88,83,100,4.61,FJ,1732486838
3,3,botou,38.0667,116.5667,44.22,90,100,5.39,CN,1732486839
4,4,punta arenas,-53.1500,-70.9167,48.31,53,75,9.22,CL,1732486741


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
citydata_humidity= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha= 0.7,
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["City","Humidity"]
)
# Display the map
citydata_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
idealDF = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Wind Speed"] < 20) \
    & (city_data_df["Cloudiness"] == 0)
]

#Drop any rows with null
idealDF = idealDF.dropna()

#Display Sample
idealDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,batamshinskiy,50.5594,58.2747,23.61,71,0,9.80,KZ,1732486961
113,113,olonkinbyen,70.9221,-8.7187,24.31,59,0,12.71,SJ,1732486965
148,148,qarazhal,48.0078,70.7903,26.44,76,0,12.30,KZ,1732487005
193,193,vadso,70.0744,29.7487,26.76,86,0,5.75,NO,1732487056
470,470,kamiiso,41.8167,140.6500,25.18,93,0,6.91,JP,1732487374


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
109,batamshinskiy,KZ,50.5594,58.2747,71,
113,olonkinbyen,SJ,70.9221,-8.7187,59,
148,qarazhal,KZ,48.0078,70.7903,76,
193,vadso,NO,70.0744,29.7487,86,
470,kamiiso,JP,41.8167,140.6500,93,
531,yasnyy,RU,51.0333,59.8742,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
import requests

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}
print(params)

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude= row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

{'categories': 'accommodation.hotel', 'apiKey': '33877823910a4088a3179362b0641400', 'limit': 1}
Starting hotel search
batamshinskiy - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
qarazhal - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
kamiiso - nearest hotel: 旅館やまに
yasnyy - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
109,batamshinskiy,KZ,50.5594,58.2747,71,No hotel found
113,olonkinbyen,SJ,70.9221,-8.7187,59,No hotel found
148,qarazhal,KZ,48.0078,70.7903,76,No hotel found
193,vadso,NO,70.0744,29.7487,86,Scandic Vadsø
470,kamiiso,JP,41.8167,140.6500,93,旅館やまに
531,yasnyy,RU,51.0333,59.8742,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
idealPlot= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha= 0.7,
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["City", "Country", "Hotel Name"]
)
# Display the map
idealPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)